In [ ]:
import re
import time

import pandas as pd
import matplotlib.pyplot as plt
import MulticoreTSNE
import sklearn.decomposition
import tensorflow as tf

import config
import models.autoencoder
import models.cyclegan
import plot_loss_history
import util

plt.style.use('bmh')

%reload_ext autoreload
%autoreload 2

## Plot losses.

In [ ]:
model_key = '2020-05-22T23:58:01'

plot_loss_history.plot_losses(model_key, min_epoch=2, log=True, max_epoch=50)

In [ ]:
model_key = '2020-05-26T22:40:22'

plot_loss_history.plot_losses(model_key, min_epoch=2, log=True)

In [ ]:
model_key = '2020-05-27T03:21:01'

plot_loss_history.plot_losses(model_key, min_epoch=1, log=True)

In [ ]:
model_key = '2020-05-27T18:01:17'

plot_loss_history.plot_losses(model_key, min_epoch=1, log=True)

In [ ]:
model_key = '2020-05-27T19:09:56'

plot_loss_history.plot_losses(model_key, min_epoch=1, log=True)

In [ ]:
model_key = '2020-05-28T04:07:10'

plot_loss_history.plot_losses(model_key, min_epoch=50)

In [ ]:
model_key = '2020-05-28T12:28:08'

plot_loss_history.plot_losses(model_key, min_epoch=50)

In [ ]:
model_key = '2020-05-28T17:09:11'

plot_loss_history.plot_losses(model_key, min_epoch=50)

In [ ]:
model_key = '2020-05-28T18:37:49'

plot_loss_history.plot_losses(model_key, min_epoch=50)

In [ ]:
model_key = '2020-05-29T07:18:15'

plot_loss_history.plot_losses(model_key, min_epoch=50)

## Inspect model architecture.

In [ ]:
model_config = config.cyclegan_config_from_json('panzer_schaeferhund.json')
model = models.cyclegan.CycleGAN(model_config)

In [ ]:
model.D['A'].summary()

In [ ]:
util.plot_model_svg(model.D_A, dpi=50, expand_nested=True, show_shapes=True)

In [ ]:
model.G_single['A'].summary()

In [ ]:
util.plot_model_svg(model.G_single['A'], dpi=50, show_shapes=True)

In [ ]:
model.G.summary()

In [ ]:
model_config = config.autoencoder_config_from_json('panzer_schaeferhund_autoencoder_adain.json')
model = models.autoencoder.AutoEncoder(model_config)

print(model.encoder.count_params())
print(model.decoder.count_params())

display(util.plot_model_svg(model.encoder, dpi=40, show_shapes=True))
display(util.plot_model_svg(model.decoder, dpi=40, show_shapes=True))

In [ ]:
model_config = config.autoencoder_config_from_json('panzer_schaeferhund_autoencoder_cnn_both.json')
model = models.autoencoder.AutoEncoder(model_config)

model.encoder.summary()
print(model.decoder.count_params())

display(util.plot_model_svg(model.encoder, dpi=40, show_shapes=True))
display(util.plot_model_svg(model.decoder, dpi=40, show_shapes=True))

## Inspect autoencoded images

In [ ]:
def plot_encoded(model_key):
    result_paths = config.construct_result_paths(model_key=model_key)

    REGEX_FOLD_DOMAIN = re.compile('(train|test)_([AB])')
    REGEX_FOLD_DOMAIN.findall(to_save['path'][0])

    def extract_domain(path_str):
        _, domain = REGEX_FOLD_DOMAIN.findall(path_str)[0]
        return domain

    encoded = pd.read_parquet(result_paths.encoded_images['A'] / 'encoded.parquet')
    encoded['domain'] = encoded['path'].apply(extract_domain)

    stds_to_plot = 3
    z = util.pandas_series_to_numpy(encoded['z'])

    def scatterplot(method):
        _, ax = plt.subplots(figsize=(10, 10))
        z = util.pandas_series_to_numpy(encoded[f'z_{method}'])
        x, y = z[:, 0], z[:, 1]
        x_min = x.mean() - stds_to_plot * x.std()
        x_max = x.mean() + stds_to_plot * x.std()
        y_min = y.mean() - stds_to_plot * y.std()
        y_max = y.mean() + stds_to_plot * y.std()
        for domain in config.DOMAINS:
            z = util.pandas_series_to_numpy(encoded[encoded['domain'] == domain][f'z_{method}'])
            x, y = z[:, 0], z[:, 1]
            ax.scatter(
                x=x,
                y=y,
                label=domain,
                alpha=0.5,
                s=0.5
            )
        ax.legend()
        ax.set_xlim((x_min, x_max))
        ax.set_ylim((y_min, y_max))
        ax.set_title(method)
        plt.savefig(result_paths.encoded_images['A'] / f'{method}.pdf')

    pca = sklearn.decomposition.PCA(n_components=2)
    z_pca = pca.fit_transform(z)
    encoded['z_pca'] = list(z_pca)
    scatterplot('pca')
    
    print("computing TSNE ...")
    start = time.time()
    tsne = MulticoreTSNE.MulticoreTSNE(
        n_components=2, random_state=42, n_jobs=8
    )
    z_tsne = tsne.fit_transform(z)
    encoded['z_tsne'] = list(z_tsne)
    print(f"... finished in {time.time() - start:.0f}s")
    scatterplot('tsne')

In [ ]:
plot_encoded('2020-05-29T11:23:02')

In [ ]:
plot_encoded('2020-05-29T14:41:58')